In [22]:
import selenium.webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
import json
import csv
import requests

C:\Users\janva\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")
driver.get("https://www.gall.nl/valdivieso-winemakers-reserva-cabernet-sauvignon-rood-75cl-387568.html")

<ipython-input-3-d5ccd4dc3edc>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")


In [4]:
res = driver.page_source.encode('utf-8')
soup = BeautifulSoup(res, "html.parser")

In [27]:
def get_soup(url):
    #driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")
    #driver.get(url)
    #res = driver.page_source.encode('utf-8')
    #soup = BeautifulSoup(res, "html.parser")
    r = requests.get(url, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(r.text, "html.parser")
    return soup

In [23]:
r = requests.get("https://www.gall.nl/valdivieso-winemakers-reserva-cabernet-sauvignon-rood-75cl-387568.html", headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
soup = BeautifulSoup(r.text, "html.parser")
soup


<!DOCTYPE html>

<html class="no-js is--v2" data-edit-mode="false" dir="ltr" lang="nl">
<head>
<meta charset="utf-8"/>
<script src="https://cdn.optimizely.com/js/19212021254.js"></script>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="/on/demandware.static/Sites-gall-nl-Site/-/nl_NL/v1678361805026/js-external/cookie-consent.min.js" name="cookie-script"/>
<meta content="" name="client-log-level"/>
<script data-mopinion-deployment-id="2o8m5cnx2oxb82ji8ign87czqii5jwgsrwv" defer="" id="mopinion-script" src="//deploy.mopinion.com/js/pastease.js"></script>
<meta content="Valdivieso Winemaker's Reserva Cabernet Sauvignon - 75CL kopen? | Gall &amp; Gall" property="og:title"/>
<meta content="product" property="og:type"/>
<meta content="https://static.gall.nl/images/IMG_680288_500.png?rev=0.2" property="og:image"/>
<meta content="https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-S

In [25]:
data = []
reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))[0]['review']
for review in reviews:
    info = {'link' : "asdf",
            "rating" : review['reviewRating']['ratingValue'],
            "date" : review['datePublished'],
            "review" : review['description']}
    try:
        info['type'] = soup.find('span', class_="pdp-info_profile").text.strip()
    except:
        info['type'] = None
    try:
        info['rating_average'] = float(soup.find('span', class_ ="rating_label").text)
    except:
        info['rating_average'] = None
    try:
        info['count'] = int(soup.find('span', class_ ="rating_label")['data-count'])
    except:
        info['count'] = None
    try:
        info['title'] = soup.find('h1', class_ ="pdp-info_name").text
    except:
        info['title'] = None
    try:
        info['country'] = soup.find('span', class_ = re.compile("flag-round u-flag-large u-hidden"))['title']
    except:
        info['country'] = None
    try:
        info['size'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[0].text
    except:
        info['size'] = None
    try:
        info['percentage'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[1].text
    except:
        info['percentage'] = None
    try:
        info['year'] = soup.find('p', class_ ="pdp-info_desc").find_all('span')[2].text
    except:
        info['year'] = None
    try:
        info['badge'] = soup.find('p', class_ ="pdp-info_badges").text.strip()
    except:
        info['badge'] = None
    try:
        info['price'] = float(soup.find('strong', class_ = "price-v2 pdp-info_price base-price")['aria-label'].replace('€ ', ''))
    except:
        info['price'] = None
    try:
        info['main_description'] = soup.find('div', class_ ="pdp-info_short s-rich-text").text.strip()
    except:
        info['main_description'] = None
    try:
        usps = []
        for item in soup.find('ul', class_ = "m-usp-bar").find_all('div', class_="usp__text"):
            usps.append(item.text.strip())
        info['usps'] = usps
    except:
        info['usps'] = None

    try:
        info['secondary_description'] = soup.find('div', class_ = "pdp-details-description s-rich-text").text.strip()
    except:
        info['secondary_description'] = None
    details = [e.text for e in soup.find_all('span', class_ ="pdp-details__item-label")]
    try:
        pos = details.index("Smaak")
        info['taste'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['taste'] = None
    try:
        pos = details.index("Lekker bij")
        info['complementaries'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['complementaries'] = None
    try:
        pos = details.index("Druivensoort")
        info['grape'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['grape'] = None
    try:
        pos = details.index("Etiketinformatie")
        info['label'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['label'] = None
    try:
        pos = details.index("Prijswinnend")
        info['certificate'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['certificate'] = None

    try:
        info['producer_description'] = soup.find('div', class_="background-story__description s-rich-text read-more__text").text.strip()
    except:
        info['producer_description'] = None
    try:
        info['producer'] = soup.find('span', class_="background-story__name").text
    except:
        info['producer'] = None
    data.append(info)
data

[{'link': 'asdf',
  'rating': 5,
  'date': '2023-02-10',
  'review': 'Mooie wijn voor goede prijs. Licht.',
  'type': 'Easy Rider',
  'rating_average': 4.3,
  'count': 178,
  'title': "Valdivieso Winemaker's Reserva Cabernet Sauvignon",
  'country': 'Chili',
  'size': '75CL',
  'percentage': '13,5%',
  'year': '2020',
  'badge': None,
  'price': 7.98,
  'main_description': 'De Nieuw-Zeelandse wijnmaker Brett David Jackson zorgt jaar in jaar uit voor een soepele wijn met stevige tannines. Je proeft cassis, zwarte bessen en frambozen. Rond en zacht met specerijen.',
  'usps': ['Bestel voor 22:00 vandaag, morgen in huis',
   'Gratis bezorging vanaf € 50,-',
   'Gratis ophalen in een Gall & Gall winkel'],
  'secondary_description': "Fruitaroma's van frambozen, aardbeien en zwarte bessen komen je tegemoet! Naast een subtiele lichte krudigheid. Dit is een rode wijn met een heerlijke fruitigheid en heel veel smaak.Wist je dat Valdivieso eerst uitsluitend begon met het produceren van mousseren

In [19]:
soup

<html class="js is--v2 is-touch" data-edit-mode="false" dir="ltr" lang="nl" style="overflow: hidden;"><head>
<meta charset="utf-8"/>
<script async="" src="https://www.google-analytics.com/plugins/ua/ec.js" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/plugins/ua/linkid.js" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-T662X77"></script><script src="https://cdn.optimizely.com/js/19212021254.js"></script>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="/on/demandware.static/Sites-gall-nl-Site/-/nl_NL/v1678361805026/js-external/cookie-consent.min.js" name="cookie-script"/>
<meta content="" name="client-log-level"/>
<script data-mopinion-deployment-id="2o8m5cnx2oxb82ji8ign87czqii5jwgsrwv" defer="" id="mopi

In [35]:
soup.find('p', class_ ="pdp-info_desc").find_all('span')

[<span class="pdp-info_country">
 <span class="fi fi-cl u-flag-round u-flag-large u-hidden" title="Chili"></span>
 Chili
 - Cabernet sauvignon
 </span>,
 <span class="fi fi-cl u-flag-round u-flag-large u-hidden" title="Chili"></span>,
 <span>2020</span>]

In [29]:
with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews_corrected.csv", "w", encoding = 'utf-8-sig', newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(list(info.keys()))
    #writer.writerow(list(info.values()))

In [30]:
def get_product_info(product_url, data):
    soup = get_soup(product_url)
    try:
        reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))[0]['review']

        for review in reviews:
            info = {'link' : product_url,
                    "rating" : review['reviewRating']['ratingValue'],
                    "date" : review['datePublished'],
                    "review" : review['description']}
            try:
                info['type'] = soup.find('span', class_="pdp-info_profile").text.strip()
            except:
                info['type'] = None
            try:
                info['rating_average'] = float(soup.find('span', class_ ="rating_label").text)
            except:
                info['rating_average'] = None
            try:
                info['count'] = int(soup.find('span', class_ ="rating_label")['data-count'])
            except:
                info['count'] = None
            try:
                info['title'] = soup.find('h1', class_ ="pdp-info_name").text
            except:
                info['title'] = None
            try:
                info['country'] = soup.find('span', class_ = re.compile("flag-round u-flag-large u-hidden"))['title']
            except:
                info['country'] = None
            try:
                info['size'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[0].text
            except:
                info['size'] = None
            try:
                info['percentage'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[1].text
            except:
                info['percentage'] = None
            try:
                info['year'] = soup.find('p', class_ ="pdp-info_desc").find_all('span')[2].text
            except:
                info['year'] = None
            try:
                info['badge'] = soup.find('p', class_ ="pdp-info_badges").text.strip()
            except:
                info['badge'] = None
            try:
                info['price'] = float(soup.find('strong', class_ = "price-v2 pdp-info_price base-price")['aria-label'].replace('€ ', ''))
            except:
                info['price'] = None
            try:
                info['main_description'] = soup.find('div', class_ ="pdp-info_short s-rich-text").text.strip()
            except:
                info['main_description'] = None
            try:
                usps = []
                for item in soup.find('ul', class_ = "m-usp-bar").find_all('div', class_="usp__text"):
                    usps.append(item.text.strip())
                info['usps'] = usps
            except:
                info['usps'] = None

            try:
                info['secondary_description'] = soup.find('div', class_ = "pdp-details-description s-rich-text").text.strip()
            except:
                info['secondary_description'] = None
            details = [e.text for e in soup.find_all('span', class_ ="pdp-details__item-label")]
            try:
                pos = details.index("Smaak")
                info['taste'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['taste'] = None
            try:
                pos = details.index("Lekker bij")
                info['complementaries'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['complementaries'] = None
            try:
                pos = details.index("Druivensoort")
                info['grape'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['grape'] = None
            try:
                pos = details.index("Etiketinformatie")
                info['label'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['label'] = None
            try:
                pos = details.index("Prijswinnend")
                info['certificate'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['certificate'] = None

            try:
                info['producer_description'] = soup.find('div', class_="background-story__description s-rich-text read-more__text").text.strip()
            except:
                info['producer_description'] = None
            try:
                info['producer'] = soup.find('span', class_="background-story__name").text
            except:
                info['producer'] = None
            try:
                data.append(info)
                if len(data) % 100 == 0:
                    print(len(data))
            except:
                print('could not append')

            with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews_corrected.csv", "a", encoding = 'utf-8-sig', newline = '') as csv_file:
                writer = csv.writer(csv_file, delimiter = ";")
                writer.writerow(list(info.values()))
    except:
        print(f"There are no reviews for {product_url}")
        time.sleep(2)


In [31]:
data

[{'link': 'product_url',
  'rating': 4.3,
  'date': '2023-02-10',
  'review': 'Mooie wijn voor goede prijs. Licht.',
  'type': 'Easy Rider',
  'count': 178,
  'title': "Valdivieso Winemaker's Reserva Cabernet Sauvignon",
  'country': 'Chili',
  'size': '75CL',
  'percentage': '13,5%',
  'year': '2020',
  'badge': None,
  'price': 7.98,
  'main_description': 'De Nieuw-Zeelandse wijnmaker Brett David Jackson zorgt jaar in jaar uit voor een soepele wijn met stevige tannines. Je proeft cassis, zwarte bessen en frambozen. Rond en zacht met specerijen.',
  'usps': ['Bestel voor 22:00 vandaag, morgen in huis',
   'Gratis bezorging vanaf € 50,-',
   'Veilig betalen'],
  'secondary_description': None,
  'taste': 'rijp,cassisen houtrijping',
  'complementaries': 'rundvlees ,lamsvleesen kaas',
  'grape': 'Cabernet sauvignon',
  'label': 'Bevat sulfur dioxide and sulfits',
  'certificate': '2 keer bekroond',
  'producer_description': 'Valdivieso is als eerste wijnhuis in Zuid-Amerika in 1879 begon

In [9]:
reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))

In [10]:
for review in reviews[0]['review']:
    info = {
    "rating" : review['reviewRating']['ratingValue'],
    "date" : review['datePublished'],
    "description" : review['description']}
    print(info)

{'rating': 5, 'date': '2023-02-10', 'description': 'Mooie wijn voor goede prijs. Licht.'}
{'rating': 4, 'date': '2023-02-08', 'description': 'Goede prijs, kwaliteit verhouding.'}
{'rating': 4, 'date': '2022-11-30', 'description': 'Is mijn huiswijn: met kaas e.d. lekkere wijn.'}
{'rating': 4, 'date': '2022-10-08', 'description': 'Prima, zoals vanouds zeer vertrouwd'}
{'rating': 4, 'date': '2022-10-05', 'description': 'Heerlijke Chileense wijn voor een zeer schappelijke prijs met een goede  smaak. Lekker bij kaas,tapas,pasta gerechten,of gewoon als borrelwijn'}
{'rating': 4, 'date': '2022-10-01', 'description': 'Mooie wijn.'}
{'rating': 4, 'date': '2022-09-24', 'description': 'Lekkere drinkbare wijn'}
{'rating': 5, 'date': '2022-05-04', 'description': 'Fijne smaak'}
{'rating': 4, 'date': '2022-04-23', 'description': 'Hele lekker wijn, voor bij rood vlees, barbecue en ook om zo te drinken.'}
{'rating': 4, 'date': '2022-04-19', 'description': 'Meerdere keren gekocht en nooit teleurgesteld.

In [12]:
data = []
get_product_info("https://www.gall.nl/valdivieso-winemakers-reserva-cabernet-sauvignon-rood-75cl-387568.html", data)
data

[{'link': 'https://www.gall.nl/valdivieso-winemakers-reserva-cabernet-sauvignon-rood-75cl-387568.html',
  'rating': 4.3,
  'date': '2023-02-10',
  'review': 'Mooie wijn voor goede prijs. Licht.',
  'type': 'Easy Rider',
  'count': 178,
  'title': "Valdivieso Winemaker's Reserva Cabernet Sauvignon",
  'country': 'Chili',
  'size': '75CL',
  'percentage': '13,5%',
  'year': '2020',
  'badge': None,
  'price': 7.98,
  'main_description': 'De Nieuw-Zeelandse wijnmaker Brett David Jackson zorgt jaar in jaar uit voor een soepele wijn met stevige tannines. Je proeft cassis, zwarte bessen en frambozen. Rond en zacht met specerijen.',
  'usps': ['Bestel voor 22:00 vandaag, morgen in huis',
   'Gratis bezorging vanaf € 50,-',
   'Gratis ophalen in een Gall & Gall winkel'],
  'secondary_description': None,
  'taste': 'rijp,cassisen houtrijping',
  'complementaries': 'rundvlees ,lamsvleesen kaas',
  'grape': 'Cabernet sauvignon',
  'label': 'Bevat sulfur dioxide and sulfits',
  'certificate': '2 k

In [31]:
product_pages = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/gall_product_links.csv", sep = ";", index_col = 0)
product_pages

,product,page_url,name,product_link,type,country,badge,price,size,rating,count,year
0,1,https://www.gall.nl/wijn/?start=0,Mucho Más Tinto,https://www.gall.nl/mucho-mas-tinto-rood-75cl-...,Easy Rider,Spanje,2e halve prijs,6.99,75CL,4.6,200.0,NaN
1,2,https://www.gall.nl/wijn/?start=0,Mucho Más Blanco,https://www.gall.nl/mucho-mas-blanco-wit-75cl-...,Fruit Lover,Spanje,2e halve prijs,6.99,75CL,4.4,199.0,NaN
2,3,https://www.gall.nl/wijn/?start=0,La Palma Chardonnay,https://www.gall.nl/la-palma-chardonnay-wit-75...,Big White,Chili,2e halve prijs,5.99,75CL,4.4,200.0,2022.0
3,4,https://www.gall.nl/wijn/?start=0,Ogio Pinot Grigio,https://www.gall.nl/ogio-pinot-grigio-wit-75cl...,Fruit Lover,Italië,2e halve prijs,6.59,75CL,4.1,188.0,2022.0
4,5,https://www.gall.nl/wijn/?start=0,Valdivieso Merlot,https://www.gall.nl/valdivieso-merlot-rood-75c...,Easy Rider,Chili,2e halve prijs,6.59,75CL,4.3,199.0,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1794,1795,https://www.gall.nl/wijn/?start=1788,Connétable de Talbot,https://www.gall.nl/connetable-de-talbot-rood-...,Royal Red,Frankrijk,2e halve prijs,32.99,75CL,NaN,NaN,2018.0
1795,1796,https://www.gall.nl/wijn/?start=1788,Umani Ronchi Cúmaro Conero Riserva,https://www.gall.nl/umani-ronchi-c%C3%BAmaro-c...,Classic Red,Italië,2e halve prijs,30.59,75CL,NaN,NaN,2016.0
1796,1797,https://www.gall.nl/wijn/?start=1800,Penfolds RWT Barossa Valley Shiraz,https://www.gall.nl/penfolds-rwt-barossa-valle...,Big Red,Australië,NaN,163.19,75CL,NaN,NaN,2016.0
1797,1798,https://www.gall.nl/wijn/?start=1800,Château Barde-Haut,https://www.gall.nl/ch%C3%A2teau-barde-haut-ro...,Royal Red,Frankrijk,2e halve prijs,54.49,75CL,NaN,NaN,2016.0


In [32]:
data_so_far = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews_corrected.csv", sep = ";")
data_so_far
links_so_far = data_so_far['link'].unique()
links_to_do = [link for link in product_pages['product_link'] if link not in links_so_far]
print(len(product_pages))
print(len(links_so_far))
print(len(links_to_do))

1799
0
1799


In [33]:
#data = []
#with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    #writer = csv.writer(csv_file, delimiter = ";")
    #writer.writerow(['link', 'rating', 'date', 'review', 'type', 'count', 'title', 'country', 'size', 'percentage', 'year', 'badge', 'price', 'main_description', 'usps', 'secondary_description', 'taste', 'complementaries', 'grape', 'label', 'certificate', 'producer_description', 'producer'])
i = 0
#driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")
for product_link in links_to_do[0:10]:
    print(product_link)
    if i % 100 == 0:
        print(round(i/len(links_to_do),4)*100)
    i += 1
    get_product_info(product_link, data)

https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html
0.0
100
200
https://www.gall.nl/mucho-mas-blanco-wit-75cl-139939.html
300
https://www.gall.nl/la-palma-chardonnay-wit-75cl-388017.html
400
https://www.gall.nl/ogio-pinot-grigio-wit-75cl-259381.html
https://www.gall.nl/valdivieso-merlot-rood-75cl-387622.html
500
https://www.gall.nl/alamos-malbec-rood-75cl-387312.html
600
https://www.gall.nl/bruce-jack-sauvignon-blanc-wit-75cl-145122.html
700
https://www.gall.nl/cap-royal-cuvee-tradition-rood-75cl-415138.html
https://www.gall.nl/la-palma-merlot-rood-75cl-388025.html
800
https://www.gall.nl/la-palma-reserva-merlot-rood-75cl-387835.html
900


In [71]:
soup = get_soup("https://www.gall.nl/lawsons-dry-hills-riesling-wit-75cl-121304.html")
reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))

<ipython-input-70-5ea85ae83707>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")


In [73]:
reviews[0]['review']

KeyError: 'review'